In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr

# Carrega o modelo
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

# Função de resposta com histórico
def responder(mensagem, historico):
    # Constrói o prompt com todo o histórico
    prompt = "<|system|>Você é um assistente útil que responde sempre em português de forma clara e objetiva.<|end|>\n"
    for user_input, resposta in historico:
        prompt += f"<|user|>{user_input}<|end|>\n<|assistant|>{resposta}<|end|>\n"
    prompt += f"<|user|>{mensagem}<|end|>\n<|assistant|>"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    saida = tokenizer.decode(outputs[0], skip_special_tokens=True)
    resposta = saida.split("<|assistant|>")[-1].strip().split("<|end|>")[0].strip()

    historico.append((mensagem, resposta))
    return "", historico

# Interface Gradio com Chatbot
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Assistente em Português com TinyLlama")
    chatbot = gr.Chatbot()
    entrada = gr.Textbox(label="Digite sua pergunta", placeholder="Ex: Quem descobriu o Brasil?")
    limpar = gr.Button("Limpar")
    estado = gr.State([])  # histórico

    entrada.submit(responder, [entrada, estado], [entrada, chatbot])
    limpar.click(lambda: ([], ""), None, [chatbot, estado])

demo.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/tmp/ipython-input-1-2900215873.py:43: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cc7e8491585313816c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
